In [1]:
from langchain_groq import ChatGroq
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from IPython.display import Image, display

In [ ]:
groq_api_key = 'gsk..................kg'
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    sentiment: str

# 1. Preprocessing Node
def preprocess(state: State) -> State:
    cleaned = state["messages"][-1].content.strip()
    state["messages"][-1].content = cleaned
    return state

# 2. Sentiment Analysis Node
def analyze_sentiment(state: State) -> State:
    msg = state["messages"][-1].content
    state["sentiment"] = "positive" if "good" in msg else "neutral"
    return state

# 3. Chatbot Node
def chatbot(state: State) -> State:
    return {"messages": llm.invoke(state['messages'])}

# 4. Logging Node
def logger(state: State) -> State:
    print(f"LOG: {state['messages'][-1].content}, Sentiment: {state.get('sentiment')}")
    return state

In [4]:
# Build the graph
builder = StateGraph(State)
builder.add_node("preprocess", preprocess)
builder.add_node("analyze_sentiment", analyze_sentiment)  # renamed
builder.add_node("chatbot", chatbot)
builder.add_node("logger", logger)

In [5]:
# Define flow
builder.add_edge(START, "preprocess")
builder.add_edge("preprocess", "analyze_sentiment")  # renamed
builder.add_edge("analyze_sentiment", "chatbot")     # renamed
builder.add_edge("chatbot", "logger")
builder.add_edge("logger", END)

In [6]:
graph = builder.compile()

In [7]:
# Draw the graph
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [9]:
# Invoke the graph
input_message = "i am kuldeep singh."
final_state = graph.invoke({"messages": ("user", input_message)})

# Print the final state
print("Final State:", final_state)
print("Chatbot's response:", final_state['messages'][-1].content)
print("Detected Sentiment:", final_state['sentiment'])

input_message_2 = "rainy weather i vrey good."
final_state_2 = graph.invoke({"messages": ("user", input_message_2)})
print("Final State 2:", final_state_2)
print("Chatbot's response 2:", final_state_2['messages'][-1].content)
print("Detected Sentiment 2:", final_state_2['sentiment'])

LOG: Hello Kuldeep Singh! 👋 

It's nice to meet you. 

Is there anything I can help you with today? 😊
, Sentiment: neutral
Final State: {'messages': [HumanMessage(content='i am kuldeep singh.', additional_kwargs={}, response_metadata={}, id='d711a173-2bc5-4c45-aae2-6717b1b5b96e'), AIMessage(content="Hello Kuldeep Singh! 👋 \n\nIt's nice to meet you. \n\nIs there anything I can help you with today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 15, 'total_tokens': 47, 'completion_time': 0.058181818, 'prompt_time': 0.001250977, 'queue_time': 0.258194712, 'total_time': 0.059432795}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2f817832-411e-4bd2-8107-dfba1e470df2-0', usage_metadata={'input_tokens': 15, 'output_tokens': 32, 'total_tokens': 47})], 'sentiment': 'neutral'}
Chatbot's response: Hello Kuldeep Singh! 👋 

It's nice to me

In [10]:
# Invoke the graph
input_message = "my frend ajay."
final_state = graph.invoke({"messages": ("user", input_message)})

# Print the final state
print("Final State:", final_state)
print("Chatbot's response:", final_state['messages'][-1].content)
print("Detected Sentiment:", final_state['sentiment'])

input_message_2 = "doesnt like winter."
final_state_2 = graph.invoke({"messages": ("user", input_message_2)})
print("Final State 2:", final_state_2)
print("Chatbot's response 2:", final_state_2['messages'][-1].content)
print("Detected Sentiment 2:", final_state_2['sentiment'])

LOG: That's nice! I'm glad you have a friend named Ajay.  

Is there anything you'd like to tell me about Ajay?  For example:

* What do you like to do together?
* What are some of his best qualities?
* Do you have a funny story about him? 


I'm always happy to hear about people's friends! 😊 
, Sentiment: neutral
Final State: {'messages': [HumanMessage(content='my frend ajay.', additional_kwargs={}, response_metadata={}, id='6686bc9c-659f-4a6d-843a-bf751de33c98'), AIMessage(content="That's nice! I'm glad you have a friend named Ajay.  \n\nIs there anything you'd like to tell me about Ajay?  For example:\n\n* What do you like to do together?\n* What are some of his best qualities?\n* Do you have a funny story about him? \n\n\nI'm always happy to hear about people's friends! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 15, 'total_tokens': 101, 'completion_time': 0.156363636, 'prompt_time': 0.001256748, 'queue_time': 0.25159167